### Create directory and copy model and vectorizer files

In [2]:
# Make a directory to contain the model
#!mkdir .\keyword-partnumber-classification

In [25]:
# copy the model to the new directory
#!copy .\keyword_partnumber_classmodel.pickle .\keyword-partnumber-classification
#!copy .\generate-term-data.pickle .\keyword-partnumber-classification
#!copy .\generate_keyword_data.py .\keyword-partnumber-classification

        1 file(s) copied.


In [5]:
import pickle
from sklearn.svm import LinearSVC
import itertools
import numpy as np
import os

class keywordpartnumberclassifier():
    # one time initialization of model and dependent objects
    def __init__(self):
        with open("generate-term-data.pickle", "rb") as f:
            self.vectorizer = pickle.load(f)
        with open("keyword_partnumber_classmodel.pickle", "rb") as f:
            self.model = pickle.load(f)
        # dummy prediction
        print(self.predict(X = np.array(["800v"]), names = ["term"]))
        
    def predict(self, X, names):
        # get search term from X
        model_input = dict(zip(names, X))
        search_term = model_input.get("term")
        # get features from search term
        features = self.vectorizer(search_term)
        features.pop(0) # remove term from feature column data
        features.pop() # remove empty space for isPartNumber
        features = np.array(features, dtype=int).reshape(1, -1) # reshape to 2D array
        prediction = self.model.predict(features)
        return [int(prediction[0])]

In [6]:
kpc = keywordpartnumberclassifier()

[0]


### Create a docker image
1. Create requirements.txt file
2. Create Dockerfile
3. Create keywordpartnumberclassifier class with prediction logic

In [3]:
%%writefile .\keyword-partnumber-classification\requirements.txt
seldon-core
scikit-learn

Overwriting .\keyword-partnumber-classification\requirements.txt


In [7]:
%%writefile .\keyword-partnumber-classification\keywordpartnumberclassifier.py
import pickle
from sklearn.svm import LinearSVC
import itertools
import numpy as np
import os

class keywordpartnumberclassifier():
    # one time initialization of model and dependent objects
    def __init__(self):
        with open("generate-term-data.pickle", "rb") as f:
            self.vectorizer = pickle.load(f)
        with open("keyword_partnumber_classmodel.pickle", "rb") as f:
            self.model = pickle.load(f)
        
    def predict(self, X, names):
        # get search term from X
        model_input = dict(zip(names, X))
        search_term = model_input.get("term")
        # get features from search term
        features = self.vectorizer(search_term)
        features.pop(0) # remove term from feature column data
        features.pop() # remove empty space for isPartNumber
        features = np.array(features, dtype=int).reshape(1, -1) # reshape to 2D array
        prediction = self.model.predict(features)
        return [int(prediction[0])]

Overwriting .\keyword-partnumber-classification\keywordpartnumberclassifier.py


In [13]:
%%writefile .\keyword-partnumber-classification\Dockerfile
FROM python:3.7-slim
WORKDIR /app

# Install python packages
COPY requirements.txt requirements.txt
RUN pip install -r requirements.txt

# Copy source code
COPY . .

# Port for GRPC
EXPOSE 5000
# Port for REST
EXPOSE 9000

# Define environment variables
ENV MODEL_NAME keywordpartnumberclassifier
ENV SERVICE_TYPE MODEL

# Changing folder to default user
RUN chown -R 8888 /app

CMD exec seldon-core-microservice $MODEL_NAME --service-type $SERVICE_TYPE

Writing .\keyword-partnumber-classification\Dockerfile


### Build docker image

In [8]:
!docker build .\keyword-partnumber-classification\ -t keywordpartnumberclassifier:2.1

#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 516B done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.7-slim
#3 ...

#4 [auth] library/python:pull token for registry-1.docker.io
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.7-slim
#3 DONE 0.9s

#5 [1/6] FROM docker.io/library/python:3.7-slim@sha256:b53f496ca43e5af6994f8e316cf03af31050bf7944e0e4a308ad86c001cf028b
#5 DONE 0.0s

#6 [internal] load build context
#6 transferring context: 1.27kB done
#6 DONE 0.0s

#7 [2/6] WORKDIR /app
#7 CACHED

#8 [3/6] COPY requirements.txt requirements.txt
#8 CACHED

#9 [4/6] RUN pip install -r requirements.txt
#9 CACHED

#10 [5/6] COPY . .
#10 DONE 0.1s

#11 [6/6] RUN chown -R 8888 /app
#11 DONE 0.4s

#12 exporting to image
#12 exporting layers
#12 exporting layers 0.1s done
#12 wri

In [9]:
!docker images | findstr keywordpartnumberclassifier

keywordpartnumberclassifier                               2.1       08edc3309dc5   5 seconds ago   573MB
keywordpartnumberclassifier                               2.0       874c953e1ac4   5 minutes ago   573MB
deneemcclain/keywordpartnumberclassifier                  1.8       671b2f74676d   3 months ago    573MB
keywordpartnumberclassifier                               1.8       671b2f74676d   3 months ago    573MB
keywordpartnumberclassifier                               1.7       9d7c6d17e34f   3 months ago    573MB
keywordpartnumberclassifier                               1.6       28c96a6c9e67   3 months ago    573MB
deneemcclain/keywordpartnumberclassifier                  1.5       d12f6d1c9df4   3 months ago    573MB
keywordpartnumberclassifier                               1.5       d12f6d1c9df4   3 months ago    573MB
deneemcclain/keywordpartnumberclassifier                  1.4       e9e8e8d74d4c   3 months ago    573MB
keywordpartnumberclassifier                            

In [15]:
# test the model returns as expected by running the image and sending a request to it
#!curl.exe -g http://localhost:2732/predict -H "Content-Type: application/json" --data @test-prediction.json

{"data":{"names":[],"ndarray":[0]},"meta":{}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    95  100    46  100    49  12806  13641 --:--:-- --:--:-- --:--:-- 31666


### Tag the docker image and push to a repository

In [16]:
# tag the desired image with a new name that you want to display in the desired repository
#!docker tag <image_name> <new_image_name_with_tag>

In [1]:
# login to docker to push to the desired repository
#!docker login

In [2]:
# push the desired image to the authenticated repository
#!docker push <image_name_and_tag>